<h1> Nous sommes PDG de Warner et nous réfléchissons au prochain projet dans lequel nous souhaitons investir</h1>

In [53]:
# Import

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns
from sklearn import linear_model

%matplotlib inline



In [54]:
# Mise en forme

rc('font', **{'family': 'sans-serif', 'sans-serif': ['Computer Modern Roman']})
params = {'axes.labelsize': 12,
          'font.size': 12,
          'legend.fontsize': 12,
          'xtick.labelsize': 10,
          'ytick.labelsize': 10,
          'text.usetex': True,
          'figure.figsize': (8, 6)}
plt.rcParams.update(params)
plt.close("all")

sns.set_context("poster")
sns.set_palette("colorblind")
sns.axes_style()
sns.set_style({'legend.frameon': True})
color_blind_list = sns.color_palette("colorblind", 8)
my_orange = color_blind_list[2]
my_green = color_blind_list[1]


<h1><FONT color="red">I. Un film, est-ce bien sérieux ? </FONT></h1>

In [56]:


# source: https://www.quandl.com/data/MOJO/BOXOFFICE_A-Yearly-Box-Office
data_movie_path='/Users/jeremieguez/Documents/MS-BGD2015/Jeremie_Guez/Exam Final/MOJO-BOXOFFICE_A.csv'
movie=pd.read_csv(data_movie_path, sep=",",encoding="utf-8")
movie['newYear']= list(map(lambda x: x.split('-')[0], movie['Year']))
#plt.plot(movie['newYear'],movie['Total Gross'])
plt.figure()
fig, ax = plt.subplots(figsize=(10, 8))
plt.plot(movie['newYear'],movie['Tickets Sold'],color='green', linestyle="-")
plt.plot(movie['newYear'][:13],movie['Tickets Sold'][:13],color='red', linestyle="-")
#plt.plot([2002,2014],[1575.7,1268.1],color='red', linestyle="--")
plt.plot([1980,2002],[1022.0,1575.7],color='green', linestyle="--")
plt.xlabel('Annee')
plt.ylabel('Nombre de tickets vends (en millions)')
plt.title("Evolution de la frequentation des salles de cinema") 


freq_1980 = 1022.0
freq_2002 = 1268.1
freq_2014 = 1575.7
coef_until_2002 =  1575.7/1022.0
coef_since_2002 = -1575.7/1268.1

#Regression linéaire pour prédire la tendance
x = movie['newYear'][:13]
y = movie['Tickets Sold'][:13]
regr = linear_model.LinearRegression()
regr.fit(x[:,np.newaxis], y)

plt.plot(x, regr.predict(x[:,np.newaxis]), color='red', linestyle="--")

baisse_since_2002 = (freq_2002 - freq_2014) / freq_2002

plt.show()



RuntimeError: LaTeX was not able to process the following string:
'lp'
Here is the full report generated by LaTeX: 



<h2 color = red><FONT color="red"><center> La frequentation des salles de cinéma a baissé de  24 pourcents depuis 2002 !!</FONT></center></h2>

<h1><FONT color="blue"> Pourquoi pas plutôt une série ?</FONT></h1>

<h2> - Vent en poupe </h2>
<h2> - Partenariat avec Netflix </h2>
<h2> - Savoir faire : Hbo  </h2>
<h2> - Focus Gotham </h2>


<h2><FONT color="blue">  Reconquerir le marché des '15-34': </FONT></h2>

In [47]:
# source: http://www.cnc.fr/web/fr/statistiques
data_frequentation_path="/Users/jeremieguez/Documents/MS-BGD2015/Jeremie_Guez/Exam Final/DemographieCinema_cnc.csv"
frequentation = pd.read_csv(data_frequentation_path, sep=",",encoding="utf-8", )
frequentation_df = pd.DataFrame(frequentation)

plt.figure()
fig, ax = plt.subplots(figsize=(10, 8))

data_freq = frequentation_df.transpose()
data_freq = data_freq.reset_index()

p1 = plt.plot(data_freq['index'],data_freq[0],label = "6 - 14 ans" )
p2 = plt.plot(data_freq['index'],data_freq[1],label = "15 - 34 ans")
p3 = plt.plot(data_freq['index'],data_freq[2],label = "+35 ans")
plt.legend(loc='upper left')
plt.xlabel('Annee')
plt.ylabel('Frequentation des salles de cinema (en base 100)')
plt.title("Frequentation des salles de cinema en France en fonction de l'age") 
plt.show
#handles, labels = ax.get_legend_handles_labels()
#ax.legend(handles, labels)
#plt.legend(handles=[p1, p2, p3])
#plt.legend([p1, p2, p3], ["6 - 14 ans", "15 - 34 ans", "35 ans et plus"])

<function matplotlib.pyplot.show>

RuntimeError: LaTeX was not able to process the following string:
'lp'
Here is the full report generated by LaTeX: 



<h1><FONT color="blue"> II. Comment s'assurer le succès ? </FONT></h1>


<h2> a.) Quel genre ? </h2>
<h2> b.) Quels acteurs ?  </h2>
<h2> c.) Quel réalisateur ? </h2>

Methodologie :
Pas données financières sur la rentabilité de tel ou tel acteur / réalisateur / genre
Hypothèse : milieu très compétitif, corrélation forte entre rentabilité et nombre d'apparation
Du coup, critère retenu : acteur / réalisateur / genre le plus présent dans les 500 séries les plus populaires

In [48]:
# Crawling de IMDB pour récupérer des informations sur acteurs, genres, réalisateurs, des séries les plus populaires

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

def GetSoupFromUrl(url):
    request = requests.get(url)
    return BeautifulSoup(request.text, 'html.parser')

def infoTvshow(url):

    soup = GetSoupFromUrl(url)
    bloc_tvshow= soup.find('table', { 'id' : 'title-overview-widget-layout' })
    tvshows= bloc_tvshow.find('td', { 'id' : 'overview-top' })
    #creators= tvshows.find('div', { 'itemprop' : 'creator' }).text.split('\n')[2:]
    if not tvshows.find('div', { 'itemprop' : 'creator' }) is None:
        creators=map(lambda y: y.split(',')[0],tvshows.find('div', { 'itemprop' : 'creator' }).text.split('\n')[2:])[:-1]
    else:
        creators=[]
    if not tvshows.find('div', { 'itemprop' : 'actors' }) is None:
        actors=map(lambda y: y.split(',')[0],map(lambda x: x.split('\n'),tvshows.find('div', { 'itemprop' : 'actors' }).text.split('|')[:1])[0][2:])
    else:
        actors=[]
    return creators,actors

def bestTvshow():
    title=[]
    year=[]
    genre=[]
    time=[]
    urls=[]
    actor=[]
    creator=[]

    for i in range(1,501,50):
        
        url='http://www.imdb.com/search/title?start='+str(i)+'&title_type=tv_series'
        soup = GetSoupFromUrl(url)
        bloc_tvshow= soup.find('table', { 'class' : 'results' })
        tvshows= bloc_tvshow.findAll('tr')
        for tvshow in tvshows:
            #print note.find('strong')['title']

            tvs=tvshow.findAll('td', { 'class' : 'title' })
            for tv in tvs:
                if not (tv.find('a') is None or tv.find('span', { 'class' : 'year_type' }) is None 
                        or tv.find('span', { 'class' : 'genre' }) is None or tv.find('span', { 'class' : 'runtime' }) is None
                        or tv.find('a') is None):

                    urls.append('http://www.imdb.com/'+tv.find('a')['href'])
                    title.append(tv.find('a').text)
                    year.append(tv.find('span', { 'class' : 'year_type' }).text.split(' ')[0][1:5])
                    genre.append(tv.find('span', { 'class' : 'genre' }).text.split('|'))
                    time.append(tv.find('span', { 'class' : 'runtime' }).text.split(' ')[0])
                    creator.append(infoTvshow('http://www.imdb.com/'+tv.find('a')['href'])[0])
                    actor.append(infoTvshow('http://www.imdb.com/'+tv.find('a')['href'])[1])

                    
    year = pd.DataFrame(year,columns=['year'])
    title=pd.DataFrame(title,columns=['title'])
    genre=pd.DataFrame(genre)
    actor=pd.DataFrame(actor)
    creator=pd.DataFrame(creator)
    urls=pd.DataFrame(urls,columns=['url'])
    time=pd.DataFrame(time,columns=['time'])
   
    
    result = pd.concat([title,creator,actor,time,year,genre], axis=1)
    return result

res_bestTvshow=[]
res_bestTvshow=bestTvshow()

res_bestTvshow.columns=(['title','creator1','creator2','creator3','creator4','creator5','creator6','creator7','creator8','creator9','creator10','creator11','actor1','actor2','actor3','time','year','genre1','genre2','genre3','genre4','genre5','genre6','genre7','genre8','genre9','genre10'])

res_bestTvshow.to_csv('resultsallTvShow.csv',encoding='utf8')



AttributeError: 'NoneType' object has no attribute 'find'

In [ ]:
# source: IMDB
data_imdb_path='/Users/jeremieguez/Documents/MS-BGD2015/Jeremie_Guez/Exam Final/resultsallTvShow.csv'
imdb = pd.read_csv(data_imdb_path, sep=",",encoding="utf-8", )
imdb_df = pd.DataFrame(imdb)

imdb_df = imdb_df.fillna('Na')
imdb_df['count']=np.ones(imdb_df.shape[0])

for col in  ['genre1','genre2','genre3','creator1','creator2','actor1','actor2','actor3']:
    imdb_df[col]=imdb_df[col].map(lambda x: x.lstrip(' ').rstrip(' '))

def mostpopularindex(typeindex):   

    gb=imdb_df.groupby(imdb_df[typeindex])
    imdb_df1=gb.sum().astype('int')

    bestindex= imdb_df1.reset_index()[[typeindex,'count']]
    
    return bestindex.sort('count', ascending=False)[bestindex[typeindex] !='Na']


<h2><FONT color="blue"> a.) Choix du genre : </FONT></h2>

In [ ]:
def camembertindex(index):
    name = [x for x in mostpopularindex(index)[index][:5]]
    data =  [x for x in mostpopularindex(index)['count'][:5]]

    explode=(0.15, 0.15, 0, 0,0)
    plt.pie(data, explode=explode, labels=name, autopct='%1.1f%%', startangle=90, shadow=True)
    plt.axis('equal')
    plt.show()
    
for index in  ['genre1','genre2','genre3']:
    camembertindex(index)

<h2><FONT color="blue"> b.) Choix des acteurs : </FONT></h2>

In [ ]:
for index in  ['actor1','actor2','actor3']:
    print mostpopularindex(index)[:5]
    print '****'


<h2><FONT color="blue"> c.) Choix de l'équipe de réalisation' : </FONT></h2>

In [ ]:
for index in  ['creator1','creator2','creator3']:
    print mostpopularindex(index)[:5]
    print '****'


<h1><FONT color="blue"> CONCLUSION: </FONT></h1>

Après une étude de notre dataLab, nous avons décidé de lancer une nouvelle série, très prometteuse, avec pour acteur principal "David Duchovny", secondé par "Martin Freeman" et "Katey Sagal". Cette série sera un savant mélange de Dram, Comedy, avec une pointe de Crime.
Nous avons choisi pour l'équipe de réalisation d'associer Darren Star, Carol Mendelsohn et Anthony E. Zuiker.

Sortie prévue début 2017… A vos Netflix :)